In [1]:
import numpy as np

def matrix_minimum(matrix):
    min_index_i = 0
    min_index_j = 0
    minimum = float('inf')
    
    for i in range(len(matrix)):
        temp = matrix[i]
        min_value = np.min(temp[np.nonzero(temp)])
        j = temp.tolist().index(min_value)

        if min_value < minimum: 
            min_index_i = i
            min_index_j = j
            minimum = min_value
   
    return min_index_i, min_index_j

def compute_newick_format(dictionary, root):
    stack = []
    result = []
    stack.append(root)
    while stack:        
        current = stack.pop()
        if isinstance(current, float):
            if isinstance(current_prev, float):
                result.pop()
                result.append(")")
                
            result.append(":%.2f" % current)
            result.append(",")
            
        elif current in dictionary.keys():        
            stack.append(dictionary[current][0])
            stack.append(dictionary[current][1])
            stack.append(dictionary[current][2])
            stack.append(dictionary[current][3])
            result.append("(")
        else:
            result.append(current)
            
        current_prev = current
        
    result.pop()
    result.append(")")  
    return ''.join(result) + ';'

# WPGMA

In [2]:
def wpgma(matrix, leaves):
    matrix = np.asarray(matrix, dtype=np.float)
    leaves = leaves.copy()
    length = len(matrix)
    dictionary={}
    
    while(length > 1):
        min_index_i, min_index_j = matrix_minimum(matrix)
        
        leaves.append(leaves[min_index_i] + leaves[min_index_j])
        distance = matrix[min_index_i][min_index_j] / 2
        
        if leaves[min_index_i] not in dictionary.keys():
            distance1 = distance
        else:
            distance1 = distance - dictionary[leaves[min_index_i]][4]
        
        if leaves[min_index_j] not in dictionary.keys():
            distance2 = distance
        else:
            distance2 = distance - dictionary[leaves[min_index_j]][4]
            
        dictionary[leaves[-1]] = [distance1, leaves[min_index_i], 
                                  distance2, leaves[min_index_j], 
                                  distance]
        
        matrix = np.insert(matrix, length, values=0, axis=0)
        matrix = np.insert(matrix, length, values=0, axis=1)

        for i in range(length):
            matrix[i][-1] = (matrix[i][min_index_i] + matrix[i][min_index_j]) / 2
            matrix[-1][i] = matrix[i][-1]
        
        if min_index_i < min_index_j:
            min_index_i, min_index_j = min_index_j, min_index_i
            
        matrix = np.delete(matrix, min_index_i, 0)
        matrix = np.delete(matrix, min_index_i, 1)
        matrix = np.delete(matrix, min_index_j, 0)
        matrix = np.delete(matrix, min_index_j, 1)            
        length = len(matrix)
        del leaves[min_index_i]
        del leaves[min_index_j]
        
    return compute_newick_format(dictionary, leaves[-1])

# UPGMA

In [3]:
def upgma(matrix, leaves):
    matrix = np.asarray(matrix, dtype=np.float)
    leaves = leaves.copy()
    length = len(matrix)
    dictionary={}
    
    while length > 1: 
        min_index_i, min_index_j = matrix_minimum(matrix)
        
        leaves.append(leaves[min_index_i] + leaves[min_index_j])  
        distance = matrix[min_index_i][min_index_j] / 2
        
        if leaves[min_index_i] not in dictionary.keys():
            distance1 = distance
        else:
            distance1 = distance - dictionary[leaves[min_index_i]][4]
        
        if leaves[min_index_j] not in dictionary.keys():
            distance2 = distance
        else:
            size2 = dictionary[leaves[min_index_j]][4]
            distance2 = distance - dictionary[leaves[min_index_j]][4]
            
        dictionary[leaves[-1]] = [distance1, leaves[min_index_i], 
                                  distance2, leaves[min_index_j], 
                                  distance]
        
        matrix = np.insert(matrix, length, values=0, axis=0)
        matrix = np.insert(matrix, length, values=0, axis=1)
        
        size1 = len(leaves[min_index_i])
        size2 = len(leaves[min_index_j])
        
        for i in range(length):
            matrix[i][-1] = ((size1*matrix[i][min_index_i] 
                              + size2*matrix[i][min_index_j])
                             / (size1+size2))
            
            matrix[-1][i] = matrix[i][-1]         
        
        if min_index_i < min_index_j:
            min_index_i, min_index_j = min_index_j, min_index_i
            
        matrix = np.delete(matrix, min_index_i, 0)
        matrix = np.delete(matrix, min_index_i, 1)
        matrix = np.delete(matrix, min_index_j, 0)
        matrix = np.delete(matrix, min_index_j, 1)            
        length = len(matrix)
        del leaves[min_index_i]
        del leaves[min_index_j]    
        
    return compute_newick_format(dictionary, leaves[-1])

# NJ

In [4]:
def nj(matrix, leaves):        
    matrix = np.asarray(matrix, dtype=np.float)
    leaves = leaves.copy()
    length = len(matrix)
    dictionary={}
    
    def m(a, b):
        vs = [matrix[a][j] for j in range(len(matrix)) if j != b and j != a]
        return np.mean(vs) if len(vs) > 0 else 0
    
    while length > 1: 
        new_matrix = np.asarray(
            [[matrix[i, j] - m(i, j) - m(j, i) if i != j else 0 for j in range(length)] 
             for i in range(length)], 
            dtype=np.float)
        
        min_index_i, min_index_j = matrix_minimum(new_matrix)
        
        leaves.append(leaves[min_index_i] + leaves[min_index_j])  
        distance = matrix[min_index_i][min_index_j] / 2
        
        distance1 = 0.5 * (matrix[min_index_i, min_index_j] 
                           + m(min_index_i, min_index_j) - m(min_index_j, min_index_i))
        
        distance2 = 0.5 * (matrix[min_index_i, min_index_j] 
                           + m(min_index_j, min_index_i) - m(min_index_i, min_index_j))
            
        dictionary[leaves[-1]] = [distance1, leaves[min_index_i], 
                                  distance2, leaves[min_index_j], 
                                  distance]
        
        matrix = np.insert(matrix, length, values=0, axis=0)
        matrix = np.insert(matrix, length, values=0, axis=1)
        
        for i in range(length):
            matrix[i][-1] = 0.5 * (matrix[i][min_index_i] 
                                   + matrix[i][min_index_j] 
                                   - matrix[min_index_i][min_index_j])
            
            matrix[-1][i] = matrix[i][-1]         
        
        if min_index_i < min_index_j:
            min_index_i, min_index_j = min_index_j, min_index_i
            
        matrix = np.delete(matrix, min_index_i, 0)
        matrix = np.delete(matrix, min_index_i, 1)
        matrix = np.delete(matrix, min_index_j, 0)
        matrix = np.delete(matrix, min_index_j, 1)            
        length = len(matrix)
        del leaves[min_index_i]
        del leaves[min_index_j]    
        
    return compute_newick_format(dictionary, leaves[-1])

In [5]:
# TEST1

matrix = [[ 0, 16, 16, 10],
          [16,  0,  8,  8],
          [16,  8,  0,  4],
          [10,  8,  4,  0]]

leaves = ['A', 'B', 'C', 'D']
print('WPGMA:\t%s' % wpgma(matrix, leaves))
print('UPGMA:\t%s' % upgma(matrix, leaves))
print('NJ:\t%s' % nj(matrix, leaves))

WPGMA:	(((D:2.00,C:2.00):2.00,B:4.00):3.25,A:7.25);
UPGMA:	(((D:2.00,C:2.00):2.00,B:4.00):3.00,A:7.00);
NJ:	(((D:0.50,C:3.50):0.50,B:5.50):5.25,A:5.25);


In [6]:
# TEST2

matrix = [[0, 5, 4, 7, 6, 8],
          [5, 0, 7, 10, 9, 11],
          [4, 7, 0, 7, 6, 8],
          [7, 10, 7, 0, 5, 9],
          [6, 9, 6, 5, 0, 8],
          [8, 11, 8, 9, 8, 0]]

leaves = ['A', 'B', 'C', 'D', 'E', 'F']
print('WPGMA:\t%s' % wpgma(matrix, leaves))
print('UPGMA:\t%s' % upgma(matrix, leaves))
print('NJ:\t%s' % nj(matrix, leaves))

WPGMA:	((((C:2.00,A:2.00):1.00,B:3.00):1.00,(E:2.50,D:2.50):1.50):0.50,F:4.50);
UPGMA:	((((C:2.00,A:2.00):1.00,B:3.00):0.75,(E:2.50,D:2.50):1.25):0.65,F:4.40);
NJ:	(((E:2.00,D:3.00):1.00,((B:4.00,A:1.00):1.00,C:2.00):1.00):2.50,F:2.50);
